In [1]:
import json
import pandas as pd

In [244]:
path = '/home/kai/data/shiyi/data/flight_data/'

In [245]:
train_ori = pd.read_pickle(path+'train_ori_1M.pkl')
test_ori = pd.read_pickle(path+'testX_ori_100k.pkl')

In [249]:
features_cols = list(test_ori.columns)

In [263]:
X_train_ori = train[features_cols]
y_train = train['label']
X_test_ori = test

In [2]:
config_dict = json.load(open('automl_config.json','r'))

In [3]:
train = pd.read_pickle(config_dict['train_data_url'])
test = pd.read_pickle(config_dict['test_data_url'])
y_test = np.load('/home/kai/data/shiyi/data/flight_data/testY_100k.npy')

In [4]:
train.shape, test.shape, y_test.shape

((1000000, 9), (100000, 8), (100000,))

In [30]:
train_more = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/train_more.pkl')
val_more = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/val_more.pkl')
test_more = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/test_more.pkl')

In [31]:
train_more.shape, val_more.shape, test_more.shape

((950000, 37), (50000, 37), (100000, 36))

In [88]:
train_more177 = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/train_more177.pkl')
test_more177 = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/test_more177.pkl')

In [119]:
train_more177.shape, test_more177.shape

((950000, 177), (100000, 176))

In [5]:
for col in train.columns:
    print(col, train[col].nunique())

Month 12
DayofMonth 31
DayOfWeek 7
DepTime 27
UniqueCarrier 22
Origin 291
Dest 292
Distance 10
label 2


In [6]:
features_cols = list(test.columns)
features_cols

['Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'UniqueCarrier',
 'Origin',
 'Dest',
 'Distance']

In [7]:
X_train_ordinal = train[features_cols]
y_train = train['label']
X_test_ordinal = test

In [32]:
feat_cols_more = list(set(train_more.columns.values) - set(['label']))

In [34]:
X_train_more = train_more[feat_cols_more]
y_train_more = train_more['label']
X_val_more = val_more[feat_cols_more]
y_val_more = val_more['label']
X_test_more = test_more[feat_cols_more]

In [120]:
feat_cols_more177 = list(set(train_more177.columns.values) - set(['label']))

In [121]:
X_train_more177 = train_more177[feat_cols_more177]
y_train_more177 = train_more177['label']
X_test_more177 = test_more177[feat_cols_more177]

In [122]:
X_train_more177.shape, X_test_more177.shape

((950000, 176), (100000, 176))

In [133]:
X_train_numeric = X_train_more177.drop(cat_cols, axis=1)
X_test_numeric = X_test_more177.drop(cat_cols, axis=1)

In [135]:
X_train_numeric.shape, X_test_numeric.shape

((950000, 168), (100000, 168))

In [137]:
X_train_one_hot.shape, X_test_one_hot.shape

((1000000, 692), (100000, 692))

In [154]:
X_train_ordinal['DepTime'].quantile(1)

26.0

In [147]:
cat_cols

['Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'UniqueCarrier',
 'Origin',
 'Dest',
 'Distance']

In [165]:
for col in X_train_ordinal.columns:
    print(col, X_train_ordinal[col].nunique())

Month 12
DayofMonth 31
DayOfWeek 7
DepTime 27
UniqueCarrier 22
Origin 291
Dest 292
Distance 10


In [162]:
(X_train_ordinal['DepTime'].nunique())#/len(X_train_ordinal))

27

In [177]:
np.sum(X_train_ordinal['Origin'] < np.mean(X_train_ordinal['Origin'].value_counts()/len(X_train_ordinal))*0.3 * len(X_test_ordinal)) / len(X_train_ordinal)

0.377926

In [272]:
def convert_to_rare(df, categorical_th=100, percentage_th=0.01):
    for col in df.columns:
        if df[col].nunique() > categorical_th:
            rare_value_th = np.mean(df[col].value_counts()) * percentage_th
            print('in {}, {} out of {} data ({} out of {} categories) will be converted to "RARE_VALUE"'\
                  .format(col, 
                          np.sum(df[col] < rare_value_th), 
                          len(df),
                          np.sum(df[col].value_counts() < rare_value_th), 
                          df[col].nunique()
                          )
                 )
            df.loc[df[col].value_counts()[df[col]].values < rare_value_th, col] = "RARE_VALUE"

In [273]:
import copy
X_train_ori_rare = copy.copy(X_train_ori)
X_test_ori_rare = copy.copy(X_test_ori)
X_train_ori_rare["test"]=0
X_test_ori_rare["test"]=1
temp_df = pd.concat([X_train_ori_rare,X_test_ori_rare],axis=0)
convert_to_rare(temp_df, 50, 0.02)
for i in range(temp_df.shape[1]):
    temp_df.iloc[:,i]=temp_df.iloc[:,i].astype('str')

X_train_ori_rare = temp_df[temp_df["test"]=="0"].iloc[:,:-1].values
X_test_ori_rare = temp_df[temp_df["test"]=="1"].iloc[:,:-1].values
for i in range(X_train_ori_rare.shape[1]):
    le = preprocessing.LabelEncoder()
    le.fit(temp_df.iloc[:,:-1].iloc[:, i])
    X_train_ori_rare[:, i] = le.transform(X_train_ori_rare[:, i])
    X_test_ori_rare[:, i] = le.transform(X_test_ori_rare[:, i])

in Origin, 248570 out of 1100000 data (22 out of 292 categories) will be converted to "RARE_VALUE"
in Dest, 247771 out of 1100000 data (21 out of 293 categories) will be converted to "RARE_VALUE"


In [274]:
X_train_ori_rare = pd.DataFrame(X_train_ori_rare)
X_test_ori_rare = pd.DataFrame(X_test_ori_rare)

In [275]:
for col in X_train_ori_rare.columns:
    print(col, X_train_ori_rare[col].nunique())

0 12
1 31
2 7
3 27
4 22
5 270
6 272
7 10


In [243]:
convert_to_rare(X_train_ordinal, 100, 0.01)

in Origin, 92951 out of 1000000 data (13 out of 291 categories) will be converted to "rare_value"
in Dest, 92483 out of 1000000 data (12 out of 292 categories) will be converted to "rare_value"


In [227]:
col = 'Dest'
np.sum(X_train_ordinal[col].value_counts() < np.mean(X_train_ordinal[col].value_counts()/len(X_train_ordinal))* 0.2 * len(X_train_ordinal)), X_train_ordinal[col].nunique()

(153, 292)

In [234]:
col = 'Dest'
np.mean(X_train_ordinal[col].value_counts()) * 0.03

102.73972602739725

In [174]:
X_train_ordinal['Dest'].value_counts() < 

102.73972602739725

In [8]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import BaggingClassifier

In [9]:
l = LogisticRegression()
r = RandomForestClassifier(n_estimators=25,max_depth=10)

## One Hot Encoding

In [10]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train_ordinal)
X_train_one_hot = enc.transform(X_train_ordinal)
X_test_one_hot = enc.transform(X_test_ordinal)

In [127]:
X_train_one_hot.shape, X_test_one_hot.shape

((1000000, 692), (100000, 692))

In [128]:
r.fit(X_train_one_hot, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [129]:
y_pred = r.predict_proba(X_test_one_hot)[:,1]

In [130]:
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.49847782314144634 0.708727647513848


In [36]:
%%time
b_r = BaggingClassifier(r, max_samples=0.6, n_estimators=8, n_jobs=1)
b_r.fit(X_train_one_hot, y_train)
y_pred = b_r.predict_proba(X_test_one_hot)[:,1]
print(log_loss(y_test,y_pred), roc_auc_score(y_test, y_pred))

0.4989549307472209 0.7168805719707757
CPU times: user 16min 36s, sys: 48.8 s, total: 17min 25s
Wall time: 1min 57s


In [11]:
import lightgbm as lgb

In [179]:
features_cols

array(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance'], dtype=object)

In [91]:
def get_lgb_params():
    return {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_boost_round': 200, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                               # seems to work fine as the upper limit when combined with early_stopping_round
        'learning_rate': 0.05,
        'num_leaves': 15,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 20,
        'feature_fraction': 1,
        'feature_fraction_seed': 2018,
        'early_stopping_round':10,
        'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
        'bagging_fraction': 0.6,
        'bagging_seed': 2018,
        'scale_pos_weight': 1,
        'metric': 'auc' 
    }

In [15]:
# cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 'DepTime']
cat_cols = features_cols

In [16]:
cat_cols

['Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'UniqueCarrier',
 'Origin',
 'Dest',
 'Distance']

In [17]:
X_train_ordinal_catonly = X_train_ordinal[cat_cols]

In [79]:
X_train_more[cat_cols].head(2)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,7,19,2,18,21,157,133,6
1,4,3,1,12,18,79,172,4


In [80]:
X_train_more.describe()

,count__DayofMonth_Distance,count__DayOfWeek_DepTime,count__DepTime_Origin,count__DayofMonth_Dest,count__Month_Dest,count__Month_UniqueCarrier,DayOfWeek,count__UniqueCarrier_Origin,count__DayOfWeek_Distance,count__DayofMonth_DepTime,...,count__DepTime_Distance,Month,Origin,UniqueCarrier,count__UniqueCarrier_Dest,count__Month_DayOfWeek,count__DepTime_Dest,Dest,count__Month_DayofMonth,count__DepTime_UniqueCarrier
count,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,...,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000,950000.000000
mean,3246.582725,8385.870915,1198.334322,624.045591,1605.223801,6176.855249,3.947889,5580.642615,14327.599934,1899.349654,...,5932.264921,6.553812,143.320405,13.058878,5530.432294,12044.048942,1200.804435,143.988526,2747.283642,4445.233154
std,212.118614,1751.849816,1032.792583,528.621685,1355.295522,3256.365216,1.991790,6558.338506,733.625698,407.100851,...,1300.011499,3.449776,77.075084,6.550593,6537.966648,1308.822659,1090.197455,77.514678,140.912880,2621.191405
min,1850.000000,1.000000,1.000000,1.000000,1.000000,226.000000,1.000000,1.000000,11496.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,9548.000000,1.000000,1.000000,2218.000000,1.000000
25%,3231.000000,8445.000000,367.000000,208.000000,526.000000,3864.000000,2.000000,572.000000,14166.000000,1932.000000,...,5720.000000,4.000000,79.000000,7.000000,559.000000,10996.000000,354.000000,79.000000,2664.000000,2626.000000
50%,3283.000000,8942.000000,936.000000,513.000000,1332.000000,5782.000000,4.000000,2536.000000,14466.000000,2032.000000,...,6260.000000,7.000000,152.000000,14.000000,2503.000000,11667.000000,907.000000,152.000000,2769.000000,4293.000000
75%,3339.000000,9141.000000,1728.000000,906.000000,2285.000000,7118.000000,6.000000,10052.000000,14788.000000,2095.000000,...,6538.000000,10.000000,206.000000,19.000000,9778.000000,13057.000000,1696.000000,207.000000,2851.000000,5471.000000
max,3504.000000,9935.000000,4534.000000,2049.000000,5266.000000,13214.000000,7.000000,26454.000000,15244.000000,2357.000000,...,9588.000000,12.000000,291.000000,22.000000,26508.000000,14500.000000,5313.000000,292.000000,3073.000000,11540.000000


In [12]:
from sklearn.model_selection import train_test_split

In [86]:
X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_more, y_train_more, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord, categorical_feature=feat_cols_more)
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord, categorical_feature=feat_cols_more)

lgbm = lgb.train(get_lgb_params(), lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=100)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 10 rounds.
[100]	valid_0's auc: 0.771418
[200]	valid_0's auc: 0.782693
[300]	valid_0's auc: 0.787103
[400]	valid_0's auc: 0.790043
[500]	valid_0's auc: 0.792035
[600]	valid_0's auc: 0.793537
[700]	valid_0's auc: 0.794763
[800]	valid_0's auc: 0.795938
Early stopping, best iteration is:
[809]	valid_0's auc: 0.795988


In [94]:
sorted(dict(zip(lgbm.feature_name(), lgbm.feature_importance())).items(), key=lambda x: x[1], reverse=True)[:10]

[('count__DepTime_Origin', 1330),
 ('count__DepTime_Dest', 1292),
 ('count__Origin_Dest', 1060),
 ('count__DayofMonth_Origin', 1004),
 ('count__DayofMonth_Dest', 900),
 ('count__Month_Dest', 718),
 ('count__Month_Origin', 704),
 ('count__Month_DayofMonth', 553),
 ('count__DayofMonth_UniqueCarrier', 432),
 ('count__DepTime_UniqueCarrier', 411)]

In [87]:
# X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_more, y_train_more, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord, categorical_feature=cat_cols)
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord, categorical_feature=cat_cols)

lgbm1 = lgb.train(get_lgb_params(), lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=100)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 10 rounds.
[100]	valid_0's auc: 0.746131
[200]	valid_0's auc: 0.75759
[300]	valid_0's auc: 0.763052
[400]	valid_0's auc: 0.765759
[500]	valid_0's auc: 0.767486
[600]	valid_0's auc: 0.76884
[700]	valid_0's auc: 0.769767
[800]	valid_0's auc: 0.770591
[900]	valid_0's auc: 0.771485
[1000]	valid_0's auc: 0.772182
[1100]	valid_0's auc: 0.772715
[1200]	valid_0's auc: 0.773224
Early stopping, best iteration is:
[1223]	valid_0's auc: 0.773434


In [92]:
sorted(dict(zip(lgbm1.feature_name(), lgbm1.feature_importance())).items(), key=lambda x: x[1], reverse=True)[:10]

[('Origin', 4835),
 ('Dest', 4798),
 ('DayofMonth', 2801),
 ('DepTime', 1611),
 ('Month', 446),
 ('count__DepTime_Origin', 367),
 ('UniqueCarrier', 338),
 ('count__DepTime_Dest', 283),
 ('count__DepTime_Distance', 180),
 ('DayOfWeek', 144)]

In [74]:
# X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_more, y_train_more, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord)#, categorical_feature=cat_cols)
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord)#, categorical_feature=cat_cols)

lgbm2 = lgb.train(lgb_params, lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=50)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds.
[50]	valid_0's auc: 0.71702
[100]	valid_0's auc: 0.733581
[150]	valid_0's auc: 0.742689
Did not meet early stopping. Best iteration is:
[150]	valid_0's auc: 0.742689


In [93]:
sorted(dict(zip(lgbm2.feature_name(), lgbm2.feature_importance())).items(), key=lambda x: x[1], reverse=True)[:10]

[('DepTime', 287),
 ('count__DepTime_Origin', 208),
 ('Month', 133),
 ('count__Month_DayOfWeek', 109),
 ('count__DepTime_Dest', 109),
 ('count__UniqueCarrier_Origin', 99),
 ('DayofMonth', 85),
 ('count__Month_DayofMonth', 76),
 ('Distance', 70),
 ('count__DayOfWeek_Dest', 70)]

In [215]:
X_train_1hot, X_val_1hot, y_train_1hot, y_val_1hot = train_test_split(X_train_one_hot, y_train, test_size=0.3, random_state=2018)
lgb_train_1hot = lgb.Dataset(X_train_1hot, y_train_1hot)
lgb_val_1hot = lgb.Dataset(X_val_1hot, y_val_1hot)

In [214]:
X_train_1hot.shape, X_val_1hot.shape, y_train_1hot.shape, y_val_1hot.shape

((700000, 692), (300000, 692), (700000,), (300000,))

In [96]:
len(feat_cols_more177)

176

In [124]:
X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_more177, y_train_more177, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord, categorical_feature=cat_cols)
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord, categorical_feature=cat_cols)

lgbm177_1 = lgb.train(get_lgb_params(), lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=50)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 10 rounds.
[50]	valid_0's auc: 0.768922
[100]	valid_0's auc: 0.783071
[150]	valid_0's auc: 0.789543
[200]	valid_0's auc: 0.793094
Did not meet early stopping. Best iteration is:
[200]	valid_0's auc: 0.793094


In [125]:
X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_more177, y_train_more177, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord, categorical_feature=feat_cols_more177)
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord, categorical_feature=feat_cols_more177)

lgbm177_2 = lgb.train(get_lgb_params(), lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=50)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 10 rounds.
[50]	valid_0's auc: 0.784065
[100]	valid_0's auc: 0.80237
[150]	valid_0's auc: 0.809354
[200]	valid_0's auc: 0.812654
Did not meet early stopping. Best iteration is:
[200]	valid_0's auc: 0.812654


In [126]:
# X_train_ord, X_val_ord, y_train_ord, y_val_ord = train_test_split(X_train_more177, y_train_more177, test_size=0.3, random_state=2018)
lgb_train_ordinal = lgb.Dataset(X_train_ord, y_train_ord)#, categorical_feature=feat_cols_more177)
lgb_val_ordinal = lgb.Dataset(X_val_ord, y_val_ord)#, categorical_feature=feat_cols_more177)

lgbm177_3 = lgb.train(get_lgb_params(), lgb_train_ordinal, valid_sets=lgb_val_ordinal, verbose_eval=50)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds.
[50]	valid_0's auc: 0.760082
[100]	valid_0's auc: 0.773752
[150]	valid_0's auc: 0.781645
[200]	valid_0's auc: 0.786858
Did not meet early stopping. Best iteration is:
[200]	valid_0's auc: 0.786858


In [123]:
print(list(X_train_more177.dtypes))

[dtype('uint8'), dtype('uint8'), dtype('uint16'), dtype('uint16'), dtype('uint8'), dtype('uint16'), dtype('uint16'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint16'), dtype('uint8'), dtype('uint16'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint16'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint16'), dtype('uint8'), dtype('uint16'), dtype('uint8'), dtype('uint8'), dtype('uint16'), dtype('uint16'), dtype('uint16'), dtype('uint8'), dtype('uint8'), dtype('uint16'), dtype('uint8'), dtype('uint16'), dtype('uint16'), dtype('uint16'), dtype('uint8'), dtype('uint16'), dtype('uint16'), dtype('uint8'), dtype('uint16'), dtype('uint16'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint16'), dtype('uint16'), dtype('uint16'), dtype('int64'), dtype('uint16'), dtype('uint16'), dtype('uint16'), dtype('uint8'), dtype('uint16'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint1

In [108]:
for col in X_train_more177:
    X_train_more177[col] = X_train_more177[col].astype(np.int32)
    X_test_more177[col] = X_test_more177[col].astype(np.int32)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
